In [12]:
import os
import pandas as pd
from openpyxl import load_workbook

# Configuration
raw_data_folder = 'RawData'
output_file = 'Cleaned_Survey_Data.xlsx'
target_sheet = 'D1.1'
country_columns = ['BE', 'BG', 'CZ', 'DK', 'D-W', 'DE', 'D-E', 'EE', 'IE', 'EL', 
                  'ES', 'FR', 'IT', 'CY', 'LV', 'LT', 'LU', 'HU', 'MT', 'NL', 
                  'AT', 'PL', 'PT', 'RO', 'SI', 'SK', 'FI', 'SE', 'UK']

def process_files():
    all_data = {}
    processed_years = set()
    
    for filename in sorted(os.listdir(raw_data_folder)):
        if filename.endswith(('.xls', '.xlsx')):
            file_path = os.path.join(raw_data_folder, filename)
            year = os.path.splitext(filename)[0]
            
            try:
                xls = pd.ExcelFile(file_path)
                
                if target_sheet not in xls.sheet_names:
                    print(f"Warning: Sheet '{target_sheet}' not found in {filename}")
                    continue
                
                print(f"Processing {filename}...")
                
                # Read with no header first to find the real header
                df = pd.read_excel(file_path, sheet_name=target_sheet, header=None)
                
                # Find header row
                header_row = None
                for i in range(min(10, len(df))):
                    if any(str(col).strip() in country_columns for col in df.iloc[i].values):
                        header_row = i
                        break
                
                if header_row is None:
                    print(f"Could not find country headers in {filename}")
                    continue
                
                # Re-read with proper header
                df = pd.read_excel(file_path, sheet_name=target_sheet, header=header_row)
                df.columns = [str(col).strip() for col in df.columns]
                
                # Process each country column
                for country in country_columns:
                    if country in df.columns:
                        sheet_name = f"{country}-LeftRight"
                        
                        if sheet_name not in all_data:
                            all_data[sheet_name] = pd.DataFrame()
                        
                        # Only add if we don't already have this year's data
                        if year not in all_data[sheet_name].columns:
                            country_data = df[country].head(30).reset_index(drop=True)
                            all_data[sheet_name][year] = country_data
                            processed_years.add(year)
            
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
                continue
    
    return all_data, processed_years

def write_output(all_data, processed_years):
    if not processed_years:
        print("No valid data was processed. Check your input files.")
        return False
    
    # Create a new workbook with openpyxl first to ensure proper structure
    from openpyxl import Workbook
    wb = Workbook()
    wb.remove(wb.active)  # Remove default sheet
    
    # Create sheets and add data
    for sheet_name, data in all_data.items():
        if not data.empty:
            ws = wb.create_sheet(title=sheet_name)
            for r_idx, row in enumerate(data.itertuples(index=False), 1):
                for c_idx, value in enumerate(row, 1):
                    ws.cell(row=r_idx, column=c_idx, value=value)
            
            # Write headers
            for c_idx, col in enumerate(data.columns, 1):
                ws.cell(row=1, column=c_idx, value=col)
    
    # Save only if we have sheets
    if wb.sheetnames:
        wb.save(output_file)
        print(f"Successfully created {output_file} with {len(wb.sheetnames)} sheets")
        return True
    else:
        print("No sheets were created - no valid data found")
        return False

def main():
    # Process all files
    all_data, processed_years = process_files()
    
    # Write output
    if not write_output(all_data, processed_years):
        # Create empty file with one sheet if completely empty
        wb = Workbook()
        wb.save(output_file)
        print("Created empty output file as no data was processed")

if __name__ == '__main__':
    main()

Processing 2007.xls...
Processing 2010.xls...
Processing 2012.xls...
Processing 2014.xls...
Processing 2015.xls...
Processing 2016.xls...
Processing 2017.xls...
Processing 2019.xls...
Successfully created Cleaned_Survey_Data.xlsx with 29 sheets
